In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, tnrange

# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html
from sklearn.decomposition import FastICA
from sklearn.cross_decomposition import CCA

def cal_correlation_coefficient(corr,mode='max',abs=False):
    import numpy as np
    _list_mode = ['max','sum']
    if(type(corr) != type(np.array([]))): corr = np.array(corr)
    if(mode not in _list_mode): raise ValueError(f"mode can only be {_list_mode}")
    if(abs): corr = np.abs(corr)
    # print(corr)
    if(mode == 'max'): return corr.max(axis=1)
    if(mode == 'sum'): return corr.sum(axis=1)

corr_mode = 'sum'
corr_abs = True
# cal_correlation_coefficient(R[:X_chs,X_chs:X_chs+Y_chs],mode='sum',abs=False)


# Use all default parameter of FastICA
fica = FastICA()

# Now we want to perform FastICA on all EEG data we have
# We have 4 EEG signals with 16 channels and each consists of 7 components (bands) + original + highpass = 9
states = ['deep-anesthetic','low-anesthetic','recovery','rest']
components = ['','_highpass','_0_delta_theta','_1_alpha','_2_low_beta','_3_high_beta','_4_low_gamma','_5_middle_gamma','_6_high_gamma']
for state in tqdm(states, desc='states'):
    mat_eeg = scipy.io.loadmat(f'EEG/EEG_{state}.mat')
    mat_ecog = scipy.io.loadmat(f'EEG/ECoG_{state}.mat')
    # RUN FastICA on original, highpass, each component
    for component in tqdm(components,desc='components'):
        eeg = []
        for i in range(16):
            eeg.append(mat_eeg[f'{i}{component}'][0])
        eeg = np.array(eeg)

        ecog = []
        for i in range(128):
            ecog.append(mat_ecog[f'{i}{component}'][0])
        ecog = np.array(ecog)
        # print(eeg.shape) (16,300000)
        # print(ecog.shape) (128,300000)
        # (n_features, n_samples)

        # X = (n_samples, n_features)
        transformed_eeg = fica.fit_transform(eeg.T)
        # print(transformed_eeg.shape) (300000, 16)
        # (n_samples, n_components)
        assert (eeg == transformed_eeg.T).all() == False, f"You data is the same."

        R = np.corrcoef(transformed_eeg.T, ecog)
        corr = cal_correlation_coefficient(R[:16,16:16+128],mode=corr_mode,abs=corr_abs)
        # print(corr.shape) (16,)
        # Step4
        # argsort is acsding. [::-1] will revert the result
        group = []
        EVs = []
        for idx in tqdm(corr.argsort()[::-1],desc='EVs'):
            group.append(idx)
            # print(transformed_eeg.T[group].shape)
            # For each subset of EEG components, applied CCA
            ev_cca = CCA(n_components=len(group))
            ev_cca.fit(transformed_eeg.T[group].T,ecog.T)
            ev_R = np.corrcoef(ev_cca.x_scores_.T,ev_cca.y_scores_.T)
            # print(ev_R.shape)
            # print(ev_R)
            # the "sum of canonical correlation coefficients" was calculated for each freq, state, and subject
            ccc = cal_correlation_coefficient(ev_R[:len(group),len(group):2*len(group)],mode='sum',abs=True)
            EVs.append(ccc)
            # print(EVs)
            # if(len(group) == 16): break
        # print(EVs)
        scipy.io.savemat(f'BSS/FastICA_{state}{component}',EVs)
        # break
    # break


(1, 300000)
[array([0.61911332])]
(2, 300000)
[array([0.61911332]), array([0.64980456, 0.48995315])]
(3, 300000)
[array([0.61911332]), array([0.64980456, 0.48995315]), array([0.65059574, 0.58140761, 0.48904874])]
(4, 300000)
[array([0.61911332]), array([0.64980456, 0.48995315]), array([0.65059574, 0.58140761, 0.48904874]), array([0.68977543, 0.5990235 , 0.50291466, 0.23330992])]
(5, 300000)
[array([0.61911332]), array([0.64980456, 0.48995315]), array([0.65059574, 0.58140761, 0.48904874]), array([0.68977543, 0.5990235 , 0.50291466, 0.23330992]), array([0.70189805, 0.5990334 , 0.50378933, 0.26037536, 0.23127497])]
(6, 300000)
[array([0.61911332]), array([0.64980456, 0.48995315]), array([0.65059574, 0.58140761, 0.48904874]), array([0.68977543, 0.5990235 , 0.50291466, 0.23330992]), array([0.70189805, 0.5990334 , 0.50378933, 0.26037536, 0.23127497]), array([0.70997065, 0.59974069, 0.53192248, 0.30076274, 0.26047656,
       0.22792025])]
(7, 300000)
[array([0.61911332]), array([0.64980456, 0

AttributeError: 'list' object has no attribute 'items'

In [9]:
temp = scipy.io.loadmat(f'EEG/ECoG_{state}.mat')
temp.keys()

dict_keys(['__header__', '__version__', '__globals__', '0', '0_highpass', '0_0_delta_theta', '0_1_alpha', '0_2_low_beta', '0_3_high_beta', '0_4_low_gamma', '0_5_middle_gamma', '0_6_high_gamma', '1', '1_highpass', '1_0_delta_theta', '1_1_alpha', '1_2_low_beta', '1_3_high_beta', '1_4_low_gamma', '1_5_middle_gamma', '1_6_high_gamma', '2', '2_highpass', '2_0_delta_theta', '2_1_alpha', '2_2_low_beta', '2_3_high_beta', '2_4_low_gamma', '2_5_middle_gamma', '2_6_high_gamma', '3', '3_highpass', '3_0_delta_theta', '3_1_alpha', '3_2_low_beta', '3_3_high_beta', '3_4_low_gamma', '3_5_middle_gamma', '3_6_high_gamma', '4', '4_highpass', '4_0_delta_theta', '4_1_alpha', '4_2_low_beta', '4_3_high_beta', '4_4_low_gamma', '4_5_middle_gamma', '4_6_high_gamma', '5', '5_highpass', '5_0_delta_theta', '5_1_alpha', '5_2_low_beta', '5_3_high_beta', '5_4_low_gamma', '5_5_middle_gamma', '5_6_high_gamma', '6', '6_highpass', '6_0_delta_theta', '6_1_alpha', '6_2_low_beta', '6_3_high_beta', '6_4_low_gamma', '6_5_middl